In [1]:
import os
import cv2
import numpy as np

data_dir = "Hindi/Train"
categories = os.listdir(data_dir)

data = []
labels = []

for idx, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (32, 32))  # Resize to 32x32 for CNN input
            data.append(img)
            labels.append(idx)
        except:
            pass

data = np.array(data).reshape(-1, 32, 32, 1) / 255.0  # Normalize
labels = np.array(labels)

print("Categories found:", categories)
print("Total images loaded:", len(data))


Categories found: ['.ipynb_checkpoints', 'character_10_yna', 'character_11_taamatar', 'character_12_thaa', 'character_13_daa', 'character_14_dhaa', 'character_15_adna', 'character_16_tabala', 'character_17_tha', 'character_18_da', 'character_19_dha', 'character_1_ka', 'character_20_na', 'character_21_pa', 'character_22_pha', 'character_23_ba', 'character_24_bha', 'character_25_ma', 'character_26_yaw', 'character_27_ra', 'character_28_la', 'character_29_waw', 'character_2_kha', 'character_30_motosaw', 'character_31_petchiryakha', 'character_32_patalosaw', 'character_33_ha', 'character_34_chhya', 'character_35_tra', 'character_36_gya', 'character_3_ga', 'character_4_gha', 'character_5_kna', 'character_6_cha', 'character_7_chha', 'character_8_ja', 'character_9_jha', 'digit_0', 'digit_1', 'digit_2', 'digit_3', 'digit_4', 'digit_5', 'digit_6', 'digit_7', 'digit_8', 'digit_9']
Total images loaded: 78200


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])


C:\Users\pt839\anaconda3\envs\tfenv\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Training samples: 62560
Test samples: 15640


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

print("⏳ Building CNN model...")

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("✅ CNN model built and compiled successfully!")

model.summary()


⏳ Building CNN model...


C:\Users\pt839\anaconda3\envs\tfenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ CNN model built and compiled successfully!


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 47)             │         6,063 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,919 (1.22 MB)

 Trainable params: 319,919 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
print("🚀 Training started...")
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)
print("✅ Training completed!")


🚀 Training started...
Epoch 1/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 31s 15ms/step - accuracy: 0.4661 - loss: 1.9529 - val_accuracy: 0.9160 - val_loss: 0.2893
Epoch 2/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 29s 15ms/step - accuracy: 0.8214 - loss: 0.5709 - val_accuracy: 0.9435 - val_loss: 0.1840
Epoch 3/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.8749 - loss: 0.3931 - val_accuracy: 0.9579 - val_loss: 0.1367
Epoch 4/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.9028 - loss: 0.3037 - val_accuracy: 0.9641 - val_loss: 0.1156
Epoch 5/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 26s 13ms/step - accuracy: 0.9200 - loss: 0.2488 - val_accuracy: 0.9630 - val_loss: 0.1198
Epoch 6/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 26s 13ms/step - accuracy: 0.9330 - loss: 0.2102 - val_accuracy: 0.9689 - val_loss: 0.1013
Epoch 7/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.9410 - loss: 0.1780 - val_accuracy: 0.9735 - val_loss: 0.0908
Epoch 8/10
1955/1955 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/ste

In [5]:
model.save("devanagari_cnn_model.h5")
print("✅ Model saved as 'devanagari_cnn_model.h5'")


✅ Model saved as 'devanagari_cnn_model.h5'
